In [ ]:
import cv2
import pickle
from scipy.misc import imread
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0",
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

In [1]:
path_prefix = '../dataset/'
depth_gt = pickle.load(open('../pkls/SUNRGBD/Depth_v7.pkl', 'rb'))
depth_keys = sorted(depth_gt.keys())
depth_img_path = path_prefix + depth_keys[20]
depth_img = imread(depth_img_path).astype('float32')
#backtorgb = cv2.cvtColor(depth_img,cv2.COLOR_GRAY2RGB)

NameError: name 'pickle' is not defined

In [ ]:
print(depth_img)
plt.imshow(depth_img)
plt.show()
index = np.where(depth_img == 2047)